# **Assignment #2**
Due date: **October 29th, 2025, 11:59PM**


### Background

In this assignment we will examine a defined benefit pension plan's strategic asset allocation (SAA) choice using a two-risk factor model, with risk factor #1 = business cycle risk (GDP) and risk factor #2 = interest rate risk (RTS).

GDP risk is measured by the percentage change in GDP from year-to-year.

RTS is the change in a mid-maturity yield from year to year; so GDP is an economic risk measure and RTS is a measure of interest rate movement.

The pension plan is considering investing in three actively actively-managed investment funds:
Fixed Income (bonds, B), Equity (E) and Commercial Real Estate (RE). The fund can also invest in a riskles asset, T bill (cash, C).

The plan's trustees do not want to change active managers. Instead they want to determine how to allocate the plan's assets of among three possible asset class investments (i.e., the SAA decision). The plan will ignore cash, but use bonds (B), Equity (E) and Real Estate (RE).

Importantly, the pension fun **cannot have short positions** in any asset class or borrow funds.


## Data and preliminary functions

You have access to estimates of the annualized risk factor "market prices of risk", risk permiums ($\lambda_{GDP}$=5% and $\lambda_{RTS}$=-1%) and standard deviations of shocks ($\sigma_{GDP}$=4% and $\sigma_{RTS}$=1%) as well as a time series of annual return data for the two risk factor shocks ($I_{GDP}$ and $I_{RTS}$) and the four actively-managed investment funds (C, B, E, RE).

By running the following code blocks you will be able to upload the data needed to answer the questions in this assignment.


In [ ]:
# Import packages
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
# **************************************
# Factors and their volatilities
# **************************************

lambda_GDP = 0.05
lambda_RTS = -0.01

sigma_GDP = 0.04
sigma_RTS = 0.01

rf = 0.01


# Collect the data in a dataframe

lambdas = {
    'GDP': lambda_GDP,
    'RTS': lambda_RTS
}

factor_volatilities = {
    'GDP': sigma_GDP,
    'RTS': sigma_RTS

}

# create a dataframe that has as index GDP and RTS and as column lambnda and sigma
df_factors = pd.DataFrame(index=['GDP', 'RTS'], columns=['lambdas', 'factor_volatility'])

df_factors['lambdas'] = lambdas
df_factors['factor_volatility'] = factor_volatilities

# add the riskfree rate to teh dataframe
df_factors['rf'] = 1.00

df_factors

,lambdas,factor_volatility,rf
GDP,0.05,0.04,1.0
RTS,-0.01,0.01,1.0


In [ ]:
# ****************************************************************************************************
# Read in the data on the time series of returns on actively managed funds and on the factor shocks
# ***************************************************************************************************

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# load the data from the github
url = "https://raw.githubusercontent.com/BillTilford/COMM475/main/Assignment_2_data.csv"


# Skip the first line and use the second line as the header
data = pd.read_csv(url, skiprows=1)

# change the index of the dataframe to the year
data = data.set_index('year')


# create a dataframe with only the returns on Tbill, Fixed Inc, Equity, and Real Estate
df_returns = data[['Tbills', 'Fixed Inc', 'Equity', 'Real Estate']]

# create a dataframe with only the returns on GDP and RTS
df_factors = data[['GDP', 'RTS']]


# Instructions:
# 1. To extract, for example, the time series of returns on Equity, we can use the following command
# equity_return = df_returns['Equity']

# 2. To extract, for example, the time series of returns on GDP, we can use the following command
# I_GDP = df_factors['GDP']

# 3. To compute, for eample, the average return on Equity, we can use the following command
# equity_return.mean()

# 4. To compute, for example, the mean of the time series of shocks to the GDP factor, we can use the following command
# I_GDP.mean()

# you can use the mat plot library "plt" function to plot data, for example plt.plot(), plt.legend(), plt.show(), etc -leverage help in Colab to do plots
# Let's have a look at the first few lines in the dataframe data
data.head()


,GDP,RTS,Tbills,Fixed Inc,Equity,Real Estate
year,,,,,,
1990.0,-0.0156,-0.0009,0.1354,0.0684,-0.1246,0.0581
1991.0,-0.0176,-0.0219,0.0924,0.1470,0.1561,0.0449
1992.0,0.0386,-0.0040,0.0668,0.0954,0.0260,0.1470
1993.0,-0.0054,-0.0160,0.0536,0.1920,0.2877,0.0325
1994.0,0.0149,0.0163,0.0534,-0.0345,0.0131,0.0756


Finally, the following block of code define a function that will be used to run regressions

In [ ]:
# We build a function that we can use repeatedly to run regressions
def multiple_regression(X,Y):
    # Add a constant to X
    X = sm.add_constant(X)

    # Replace NaN values with the mean of the column
    X = X.fillna(X.mean())
    Y = Y.fillna(Y.mean())

    # Fit and summarize OLS model
    results = sm.OLS(Y, X).fit()
    return results

# Instructions on how to run regressions:
# To run a regresson of the returns on Equity on the returns on GDP and RTS, we can use the following commands:
#  X = df_factors[['GDP', 'RTS']]
#  Y = df_returns['Equity']
#  results = multiple_regression(X,Y)

# To see the resujlts, type
# results.summary()

# To extract the coefficients, type
# results.params

# To extract the t-stats, type
# results.tvalues

# To extract the R-squared, type
# results.rsquared


### Question 1

Explain why the risk factors shocks are called "shocks." Compute the mean of the GDP and RTS factor shocks. What do you find?Comment on the result. Explain how the shocks $I_{GDP}$ and $I_{RTS}$ in the dataframe `data` have been calculated

In [53]:

mean_GDP = df_factors['GDP'].mean()
mean_RTS = df_factors['RTS'].mean()

print("Mean GDP shock:", round(mean_GDP, 6))
print("Mean RTS shock:", round(mean_RTS, 6))


Mean GDP shock: -2.3e-05
Mean RTS shock: -0.0


'Shocks' are unexpected changes or innovations in the underlying factors.
They represent deviations from what was anticipated in the previous period.
By construction, shocks are centered around zero—so their sample means
should be approximately zero. This means that, on average, there are no
systematic positive or negative surprises in GDP or interest rate changes over the sample period.
This ensures that over time, the model attributes no persistent drift to shocks, only random fluctuations around expectations. If we found a large positive or negative mean, that would imply a systematic bias rather than random surprises.


I_GDP and I_RTS were computed as the year-over-year percentage changes
in GDP and in a mid-term interest rate, respectively:
    
    I_GDP_t = (GDP_t - GDP_{t-1}) / GDP_{t-1}

    I_RTS_t = (RTS_t - RTS_{t-1})
These computations generate a time series of unexpected deviations (hence, “shocks”) used as explanatory variables in regressions.


## Question 2

Regress the bond returns on the two risk factor shocks: GDP and RTS.
Report: Intercept and t-statistic; GDP beta and t-statistic; RTS beta and t-statistic; R-squared.

Comment on what is driving the bond returns

In [ ]:

# Step 1: Define X (factors) and Y (bond returns)
X = df_factors[['GDP', 'RTS']]
Y = df_returns['Fixed Inc']

# Step 2: Run regression using the provided function
results_bond = multiple_regression(X, Y)

# Step 3: Display regression summary
print(results_bond.summary())

# Step 4: Extract key statistics
alpha = results_bond.params['const']
alpha_t = results_bond.tvalues['const']

beta_GDP = results_bond.params['GDP']
beta_GDP_t = results_bond.tvalues['GDP']

beta_RTS = results_bond.params['RTS']
beta_RTS_t = results_bond.tvalues['RTS']

R2 = results_bond.rsquared

print("\n=== Regression Results: Bonds vs GDP & RTS ===")
print(f"Intercept (α): {alpha:.6f},  t-stat: {alpha_t:.3f}")
print(f"GDP beta:       {beta_GDP:.6f},  t-stat: {beta_GDP_t:.3f}")
print(f"RTS beta:       {beta_RTS:.6f},  t-stat: {beta_RTS_t:.3f}")
print(f"R-squared:      {R2:.3f}")


                            OLS Regression Results                            
Dep. Variable:              Fixed Inc   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     68.05
Date:                Wed, 22 Oct 2025   Prob (F-statistic):           7.10e-12
Time:                        02:33:54   Log-Likelihood:                 79.497
No. Observations:                  33   AIC:                            -153.0
Df Residuals:                      30   BIC:                            -148.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0771      0.004     19.408      0.0

The regression shows that bond returns are strongly and negatively related to RTS shocks.
When interest rates rise (positive RTS), bond prices fall, producing lower returns.

The GDP beta is small and negative but statistically insignificant — This means bond returns decline when the economy improves, consistent with bonds being defensive assets. However, the sensitivity is low.

The high R² (≈ 0.82) indicates that the two-factor model, primarily through RTS, explains most of the variation in bond returns.

In short, interest rate movement, not GDP growth, is the dominant factor that drives bond performance in this model.

### Question 3

Regress the equity returns on the two risk factor shocks.
Report: Intercept and t-statistic; GDP beta and t-statistic; RTS beta and t-statistic; R-squared
Comment on what is driving the bond returns

In [ ]:

# Step 1: Define X (factors) and Y (equity returns)
X = df_factors[['GDP', 'RTS']]
Y = df_returns['Equity']

# Step 2: Run regression using the provided function
results_equity = multiple_regression(X, Y)

# Step 3: Display regression summary
print(results_equity.summary())

# Step 4: Extract coefficients, t-stats, and R²
alpha = results_equity.params['const']
alpha_t = results_equity.tvalues['const']

beta_GDP = results_equity.params['GDP']
beta_GDP_t = results_equity.tvalues['GDP']

beta_RTS = results_equity.params['RTS']
beta_RTS_t = results_equity.tvalues['RTS']

R2 = results_equity.rsquared

print("\n=== Regression Results: Equity vs GDP & RTS ===")
print(f"Intercept (α): {alpha:.6f},  t-stat: {alpha_t:.3f}")
print(f"GDP beta:       {beta_GDP:.6f},  t-stat: {beta_GDP_t:.3f}")
print(f"RTS beta:       {beta_RTS:.6f},  t-stat: {beta_RTS_t:.3f}")
print(f"R-squared:      {R2:.3f}")


                            OLS Regression Results                            
Dep. Variable:                 Equity   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.4954
Date:                Wed, 22 Oct 2025   Prob (F-statistic):              0.614
Time:                        02:33:54   Log-Likelihood:                 18.980
No. Observations:                  33   AIC:                            -31.96
Df Residuals:                      30   BIC:                            -27.47
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0983      0.025      3.954      0.0

The regression shows low explanatory power (R² ≈ 0.03), indicating that GDP and RTS shocks together explain a very small portion of equity’s variation.

Both betas have low t-statistics, indicating no statistically significant relationship between equity returns and the two factors in this sample.

The intercept (α) is positive, capturing an average excess return consistent with an equity risk premium, but it’s largely unexplained by these two macro factors.

Therefore, Equity returns are not driven by short-term GDP or interest rate shocks. They likely depend on additional risk factors such as global risk appetite, volatility, earnings expectations, or sentiment not captured here.

### Question 4

Regress the real estate returns on the two risk factor shocks. Report: Intercept and t-statistic; GDP beta and t-statistic; RTS beta and t-statistic; R-squared Comment on what is driving the bond returns

In [ ]:

# Step 1: Define X (factors) and Y (real estate returns)
X = df_factors[['GDP', 'RTS']]
Y = df_returns['Real Estate']

# Step 2: Run regression using the provided function
results_re = multiple_regression(X, Y)

# Step 3: Display regression summary
print(results_re.summary())

# Step 4: Extract key results
alpha = results_re.params['const']
alpha_t = results_re.tvalues['const']

beta_GDP = results_re.params['GDP']
beta_GDP_t = results_re.tvalues['GDP']

beta_RTS = results_re.params['RTS']
beta_RTS_t = results_re.tvalues['RTS']

R2 = results_re.rsquared

print("\n=== Regression Results: Real Estate vs GDP & RTS ===")
print(f"Intercept (α): {alpha:.6f},  t-stat: {alpha_t:.3f}")
print(f"GDP beta:       {beta_GDP:.6f},  t-stat: {beta_GDP_t:.3f}")
print(f"RTS beta:       {beta_RTS:.6f},  t-stat: {beta_RTS_t:.3f}")
print(f"R-squared:      {R2:.3f}")



                            OLS Regression Results                            
Dep. Variable:            Real Estate   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     8.971
Date:                Wed, 22 Oct 2025   Prob (F-statistic):           0.000883
Time:                        02:33:54   Log-Likelihood:                 62.148
No. Observations:                  33   AIC:                            -118.3
Df Residuals:                      30   BIC:                            -113.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0693      0.007     10.309      0.0

Real estate returns are positively and significantly related to GDP shocks: when the economy grows faster than expected, commercial property values and rents tend to rise, boosting returns.

The interest rate sensitivity (β_RTS) is positive but statistically insignificant; this means rate changes do not systematically explain much of real estate performance in this sample.

With an R² of ~0.37, the two-factor model explains a moderate share of real estate returns, which is much more than for equities but less than for bonds.

Therefore, Real estate is primarily driven by economic (GDP) conditions, reflecting its link to business cycles and growth, while interest rates play a secondary role.

### Question 5

Use the GDP betas and RTS betas in questions 2, 3, and 4 above to calculate each asset class' expected returns. Express your answers in percent, with two decimals (e.g, 5.13%). Do these seem reasonable relative to one another, given the riskiness of each asset class?

In [54]:

lambda_GDP = 0.05
lambda_RTS = -0.01

rf = 0.01

# Step 1: Extract betas from previous regressions
beta_GDP_B = results_bond.params['GDP']
beta_RTS_B = results_bond.params['RTS']

beta_GDP_E = results_equity.params['GDP']
beta_RTS_E = results_equity.params['RTS']

beta_GDP_RE = results_re.params['GDP']
beta_RTS_RE = results_re.params['RTS']

# Step 2: Compute expected return for each asset
def expected_return(beta_GDP, beta_RTS):
    return rf + beta_GDP*lambda_GDP + beta_RTS*lambda_RTS

E_B = expected_return(beta_GDP_B, beta_RTS_B)
E_E = expected_return(beta_GDP_E, beta_RTS_E)
E_RE = expected_return(beta_GDP_RE, beta_RTS_RE)

# Step 3: Display results
print("\n=== Expected Returns (Factor Model) ===")
print(f"Bonds (Fixed Income): {E_B*100:.2f}%")
print(f"Equity:               {E_E*100:.2f}%")
print(f"Real Estate:          {E_RE*100:.2f}%")



=== Expected Returns (Factor Model) ===
Bonds (Fixed Income): 5.46%
Equity:               8.65%
Real Estate:          6.98%


The ranking Equity > Real Estate > Bonds makes intuitive sense:

Equities are most exposed to business-cycle risk, so they should earn the highest expected return.

Real Estate has positive GDP sensitivity but less volatility, so its return is between bonds and equity.

Bonds, being safer and negatively related to RTS shocks (benefiting from falling rates), earn the lowest expected return among risky assets.

This hierarchy is consistent with both risk theory and market intuition — higher systematic risk, higher expected return.

### Question 6
Compute the  actual standard deviations of returns on each of the three active funds (bonds, equity and real estate) to two percent decimal point (e.g., 13.65%). Comment on the volatilities on the three funds, B, E and RE.

In [58]:

# Step 1: Calculate sample standard deviations of annual returns
std_B = df_returns['Fixed Inc'].std()
std_E = df_returns['Equity'].std()
std_RE = df_returns['Real Estate'].std()

# Step 2: Display results (in % with two decimals)
print("\n=== Standard Deviations of Returns (Volatility) ===")
print(f"Bonds (Fixed Income): {std_B*100:.2f}%")
print(f"Equity:               {std_E*100:.2f}%")
print(f"Real Estate:          {std_RE*100:.2f}%")



=== Standard Deviations of Returns (Volatility) ===
Bonds (Fixed Income): 5.37%
Equity:               14.51%
Real Estate:          4.88%


Equity has the highest volatility (≈ 14.51%), consistent with its exposure to market and economic fluctuations.
Bonds are much more stable (≈ 5.37%), as fixed income returns move mainly with predictable rate changes.
Real Estate shows the lowest volatility (≈ 4.88%), which may reflect smoother valuations or appraisal-based data compared to market-traded securities.
The volatility ranking Equity > Bonds > Real Estate reinforces the risk–return relationship seen in Question 5: higher volatility assets command higher expected returns.

### Question 7

Assume the following correlation matrix for Bond, Equity, and Real Estate returns:


|        | Bond | Equity | Real Estate |
|--------|------|--------|-------------|
| **Bond**   | 1.0    | 0.2    | 0.3         |
| **Equity** | 0.2  | 1.0      | 0.3         |
| **Real Estate** | 0.3  | 0.3    | 1.0       |

Use the expected asset class returns from question 5 and the standard deviations of returns from question 6, along with the above correlation matrix, to find and report the portfolio expected returns, standard deviations of returns and portfolio weights for portfolio with expected returns of: .065, .070, .075 and .080.
You can use the minimize function imported in the code for optimizations, or go back to Assigment 1 and import and leverage those functions and code for matrix and optimization (ie PyPortfolioOpt)

Find the portfolios that achieve the desired target returns and compute their volatility

In [61]:

import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Inputs (from Q5 and Q6)
assets = ["Bond", "Equity", "Real Estate"]
mu = np.array([0.0546, 0.0865, 0.0698])   # Bonds, Equity, Real Estate
stds = np.array([0.0537, 0.1451, 0.0488]) # Bonds, Equity, Real Estate

# Correlation matrix given in the question
corr = np.array([
    [1.0, 0.2, 0.3],
    [0.2, 1.0, 0.3],
    [0.3, 0.3, 1.0]
])

# Build covariance matrix: Sigma = D * Corr * D
D = np.diag(stds)
Sigma = D @ corr @ D

# Optimizer: minimize variance given a target return, no shorting
def min_var_given_return(target_return, mu, Sigma):
    n = len(mu)
    w0 = np.ones(n) / n

    def portfolio_var(w):
        return float(w.T @ Sigma @ w)

    cons = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type": "eq", "fun": lambda w, r=target_return: mu @ w - r},
    ]
    bounds = [(0.0, 1.0)] * n

    res = minimize(portfolio_var, w0, method="SLSQP", bounds=bounds, constraints=cons)
    if not res.success:
        raise RuntimeError(f"Optimization failed for target {target_return:.4f}: {res.message}")
    w = res.x
    er = mu @ w
    vol = np.sqrt(w.T @ Sigma @ w)
    return w, er, vol

# Targets required by the assignment (decimals)
targets = [0.065, 0.070, 0.075, 0.080]

# Solve & report
rows = []
for r in targets:
    w, er, vol = min_var_given_return(r, mu, Sigma)
    rows.append({
        "Target E[R] (%)": round(100 * r, 2),
        "Bond Weight": round(w[0], 4),
        "Equity Weight": round(w[1], 4),
        "Real Estate Weight": round(w[2], 4),
        "Portfolio E[R] (%)": round(100 * er, 2),
        "Portfolio σ (%)": round(100 * vol, 2),
    })

out = pd.DataFrame(rows, columns=[
    "Target E[R] (%)", "Bond Weight", "Equity Weight", "Real Estate Weight",
    "Portfolio E[R] (%)", "Portfolio σ (%)"
])

print("\n=== Target Portfolios ===")
print(out.to_string(index=False))



=== Target Portfolios ===
 Target E[R] (%)  Bond Weight  Equity Weight  Real Estate Weight  Portfolio E[R] (%)  Portfolio σ (%)
             6.5       0.3355         0.0179              0.6466                 6.5             4.17
             7.0       0.0749         0.0802              0.8449                 7.0             4.75
             7.5       0.0000         0.3114              0.6886                 7.5             6.39
             8.0       0.0000         0.6108              0.3892                 8.0             9.60


### Question 8
Using the results in question 7, explain how and why the portfolio compositions change as one moves up the reward-risk frontier from E(r)=.065 to E(r)=.08.

When E[R] is low (6.5% – 7.0%), Real Estate dominates the portfolio because it provides a moderate return (6.98%) with low volatility (4.88%). Bonds contribute some stability, but because their expected return (5.46%) is relatively low, the optimizer limits their weight to balance the portfolio return requirement. Equities play a very small role, as their higher volatility isn’t needed to achieve the modest target return. At this stage, the portfolios prioritize low risk over high return.

As one moves up the reward–risk frontier to E(R)=7.5% to 8.0%, the optimizer must include more of the higher-return asset (Equity). Bond exposure drops to 0, as it constrains portfolio return. Real Estate remains significant because it continues to diversify equity risk due to its low correlation with equities (ρ = 0.3). The portfolio takes on more risk to pursue higher expected returns, moving closer to the equity-dominant end of the frontier.

The overall portfolio volatility rises from 4.17% to 9.60%, illustrating the classic risk–return trade-off inherent in mean–variance efficient portfolios.

### Question 9

Mean-variance analysis, as done above, ignores the exposures of the plan's liability, and how the assets might move with or against the liability as the risk factors are shocked.

Suppose the plan has liability betas of $\beta_{GDP,L} = 1$ and $\beta_{RTS,L} = -2$.

Using the beta (exposures) of each asset class (Bond, Equity, Real Estate) to these risk factors (GDP and RTS) that you computed in questions 2,3, and 4,
calculate the optimal asset class weights that give a portfolio of only risky assets (no cash) the same betas as the liability. Explain why this weighting gives those betas.

In [70]:
import numpy as np
from scipy.optimize import minimize

# Betas from regression (Q2–Q4)
betas = {
    "Bond":       {"GDP": -0.2483, "RTS": -5.7031},
    "Equity":     {"GDP":  1.0500, "RTS": -2.3975},
    "RealEstate": {"GDP":  1.4029, "RTS":  1.0348},
}

assets = ["Bond", "Equity", "RealEstate"]

# Liability betas
beta_L = {"GDP": 1.0, "RTS": -2.0}

# Build matrices
B_GDP = np.array([betas[a]["GDP"] for a in assets])
B_RTS = np.array([betas[a]["RTS"] for a in assets])
A = np.vstack([B_GDP, B_RTS, np.ones(3)])     # 3x3
b = np.array([beta_L["GDP"], beta_L["RTS"], 1.0])

# Compute achieved betas
beta_p_gdp = B_GDP @ w
beta_p_rts = B_RTS @ w

# Print results
print("=== Liability-Matching Portfolio (No Cash) ===")
for a, wi in zip(assets, w):
    print(f"{a:11s}: {wi:6.4f}")
print(f"\nPortfolio Betas: GDP = {beta_p_gdp:+.3f},  RTS = {beta_p_rts:+.3f}")


=== Liability-Matching Portfolio (No Cash) ===
Bond       : 0.0948
Equity     : 0.6981
RealEstate : 0.2071

Portfolio Betas: GDP = +1.000,  RTS = -2.000


These weights replicate the liability’s factor sensitivities by matching the portfolio’s GDP and RTS betas to those of the liability.
Because betas combine linearly, setting ∑​wi​βGDP,i​=1, ∑​wi​βRTS,i​=−2, ∑​wi​=1 ensures that the asset portfolio moves exactly with the liability when GDP shocks occur and twice as strongly in the opposite direction when RTS shocks occur.

* GDP β = +1: requires more weight in pro-cyclical assets (equity, real estate).
* RTS β = −2: requires heavier exposure to assets that fall when rates rise (bonds and equities with negative RTS betas).

### Question 10

Using the weights from question 9, what is the portfolio expected return? Compare this portfolio composition with the portfolio derived in question 7 with the most similar expected return. How does consideration of the liability betas affect the portfolio composition?

In [72]:
import numpy as np

# Expected returns (from Q5)
mu = np.array([0.0546, 0.0865, 0.0698])   # Bonds, Equity, Real Estate

# Replace with your Q9 weights
w = np.array([0.0948, 0.6981, 0.2071])

# Portfolio expected return and volatility
E_Rp = np.dot(w, mu)
Var_p = w.T @ Sigma @ w
Std_p = np.sqrt(Var_p)

print(f"\nPortfolio Expected Return: {E_Rp*100:.2f}%")
print(f"Portfolio Volatility     : {Std_p*100:.2f}%")



Portfolio Expected Return: 8.00%
Portfolio Volatility     : 10.60%


From Q7, the 8.0% target min-variance portfolio (no shorting) was:
* Weights: Bond 0.0000, Equity 0.6108, Real Estate 0.3892
* Volatility: 9.60%

The new portfolio consists much less Real Estate (0.2071 vs 0.3892), more Equity (0.6981 vs 0.6108), and some Bonds (0.0948 vs 0.0000). It also has higher volatility (10.60% vs 9.60%)

Asset-only mean–variance portfolio (Q7) chooses mixes that minimize asset volatility, which favored Real Estate.

The liability-aware portfolio (Q9/Q10) matches the liability’s factor exposures by shifting weight from Real Estate to Equity (and some Bonds), targeting β_GDP = 1 and β_RTS = −2.

Therefore, at ~8% expected return, the liability-matching portfolio is riskier regarding asset volatility,  but should deliver lower surplus (Assets−Liability) volatility, which is the objective when liabilities matter.